In [1]:
import zipfile
from os import listdir
import numpy as np
from sklearn import preprocessing
from sklearn import decomposition
#import matplotlib.pyplot as plt
%pylab
import ConvLstm_bulk as ConvLstm

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# read historical data
filenames = listdir('./data/historical/berlin/')
temps_old = np.empty( ( len( filenames ), 24*365*65 ), dtype=np.float32 )
temps_old_sizes = []
for j, fn in enumerate( filenames ):
    with zipfile.ZipFile( './data/historical/berlin/' + fn ) as zipdata_f:
        values_filename = [zi.filename for zi in zipdata_f.filelist if zi.filename.startswith('produkt')][0]
        with zipdata_f.open( values_filename ) as data_f:
            #data_f = open( data_ff )
            header = data_f.readline()
            for i, line in enumerate( data_f.readlines() ):
                _, date, _, temp, _, _ = str(line).split(';')
                temps_old[j, i] = float( temp )
            temps_old_sizes.append(i)

['stundenwerte_TU_00400_19910101_20171231_hist.zip', 'stundenwerte_TU_00433_19510101_20171231_hist.zip', 'stundenwerte_TU_00410_20040501_20171231_hist.zip', 'stundenwerte_TU_00427_19730101_20171231_hist.zip', 'stundenwerte_TU_00430_19860101_20171231_hist.zip', 'stundenwerte_TU_00420_20070801_20171231_hist.zip', 'stundenwerte_TU_00403_20020101_20171231_hist.zip']


In [3]:
# read recent data
filenames = listdir('./data/recent/berlin/')
temps = np.empty( ( len( filenames ), 24*365*50 ), dtype=np.float32 )
temps_sizes = []
for j, fn in enumerate( filenames ):
    with zipfile.ZipFile( './data/recent/berlin/' + fn ) as zipdata_f:
        values_filename = [zi.filename for zi in zipdata_f.filelist if zi.filename.startswith('produkt')][0]
        with zipdata_f.open( values_filename ) as data_f:
            #data_f = open( data_ff )
            header = data_f.readline()
            for i, line in enumerate( data_f.readlines() ):
                _, date, _, temp, _, _ = str(line).split(';')
                temps[j, i] = float( temp )
            temps_sizes.append(i)

['stundenwerte_TU_00427_akt.zip', 'stundenwerte_TU_00410_akt.zip', 'stundenwerte_TU_00430_akt.zip', 'stundenwerte_TU_00400_akt.zip', 'stundenwerte_TU_00433_akt.zip', 'stundenwerte_TU_00403_akt.zip', 'stundenwerte_TU_00420_akt.zip']


In [4]:
np.min(temps_old_sizes)/365/24

10.396689497716896

In [5]:
batch_size = 1
seq_len = 7
time_span = 24 
space_size = temps.shape[0]
num_units = 7
num_vars = 1

clw = ConvLstm.ConvLSTMWeather( batch_size, seq_len, time_span, space_size, num_vars, kernel_shape=[12, 3] )

In [3]:
temp44 = np.empty( (13200,), dtype=np.float32 )
with open( './data/produkt_tu_stunde_20161215_20180617_00044.txt' ) as data_f:
    header = data_f.readline()
    for i, line in enumerate( data_f.readlines() ):
        _, date, _, temp, _, _ = line.split(';')
        temp44[i] = float( temp )

In [6]:
scaler = preprocessing.RobustScaler()
scaler.fit(np.reshape( temps[0,:np.min(temps_sizes)], (-1, 1) ))
temps_preproc = np.empty_like(temps)
for i in range( temps.shape[0] ):
    temps_preproc[i] = scaler.transform( np.reshape( temps[i], (-1, 1) ) )[:,0]
    
temps_old_preproc = np.empty_like(temps_old)
for i in range( temps_old.shape[0] ):
    temps_old_preproc[i] = scaler.transform( np.reshape( temps_old[i], (-1, 1) ) )[:,0]

In [7]:
def create_batch( data, batch_size, seq_len, time_span, space_size, num_vars ):
    batch_data = np.empty( ( batch_size, seq_len, time_span, space_size, num_vars ) )
    batch_labels = np.empty( ( batch_size, seq_len, time_span, space_size, num_vars ) )
    for i in range( batch_size ):
        for j in range( seq_len ):
            for k in range( space_size ):
                batch_data[i, j, :, k, 0] = data[k, i * seq_len * time_span + j * time_span : i * seq_len * time_span + (j + 1) * time_span]
                batch_labels[i, j, :, k, 0] = data[k, i * seq_len * time_span + (j + 1) * time_span: i * seq_len * time_span + (j + 2) * time_span ]
                            
    #                        .reshape( seq_len, time_span, space_size, num_vars ) )
    return batch_data, batch_labels 

In [8]:
train_data, train_labels = create_batch( temps_old_preproc, 500, seq_len, time_span, space_size, num_vars )

In [9]:
test_data, test_labels = create_batch( temps_preproc, 52, seq_len, time_span, space_size, num_vars )

In [12]:
state = clw.train( train_data, train_labels, 100 )

Step 0. Cost: 0.5706019997596741
Step 0. Cost: 0.5286294221878052
Step 0. Cost: 0.40447303652763367
Step 0. Cost: 0.4293387830257416
Step 0. Cost: 0.4433322846889496
Step 0. Cost: 0.4567236304283142
Step 0. Cost: 0.4130373001098633
Step 0. Cost: 0.5236560106277466
Step 0. Cost: 0.48918837308883667
Step 0. Cost: 0.39826810359954834
Step 0. Cost: 0.23987802863121033
Step 0. Cost: 0.2551144063472748
Step 0. Cost: 0.24392269551753998
Step 0. Cost: 0.31145811080932617
Step 0. Cost: 0.364793062210083
Step 0. Cost: 0.22587242722511292
Step 0. Cost: 0.22733597457408905
Step 0. Cost: 0.26054999232292175
Step 0. Cost: 0.17520365118980408
Step 0. Cost: 0.2590065598487854
Step 0. Cost: 0.3438166677951813
Step 0. Cost: 0.19909019768238068
Step 0. Cost: 0.2449922412633896
Step 0. Cost: 0.3676741123199463
Step 0. Cost: 0.30558887124061584
Step 0. Cost: 0.3971521556377411
Step 0. Cost: 0.440988153219223
Step 0. Cost: 0.30757567286491394
Step 0. Cost: 0.3205573558807373
Step 0. Cost: 0.3619990348815918

Step 0. Cost: 0.09180943667888641
Step 0. Cost: 0.08787296712398529
Step 0. Cost: 0.07700708508491516
Step 0. Cost: 0.11569903790950775
Step 0. Cost: 0.1416623741388321
Step 0. Cost: 0.16360950469970703
Step 0. Cost: 0.1769086718559265
Step 0. Cost: 0.1718316674232483
Step 0. Cost: 0.14505238831043243
Step 0. Cost: 0.12614355981349945
Step 0. Cost: 0.22147008776664734
Step 0. Cost: 0.1667286902666092
Step 0. Cost: 0.18372711539268494
Step 0. Cost: 0.43612053990364075
Step 0. Cost: 0.46098411083221436
Step 0. Cost: 0.4468083679676056
Step 0. Cost: 0.31543660163879395
Step 0. Cost: 0.1337849646806717
Step 0. Cost: 0.171886146068573
Step 0. Cost: 0.1489945948123932
Step 0. Cost: 0.14933159947395325
Step 0. Cost: 0.17562533915042877
Step 0. Cost: 0.19776049256324768
Step 0. Cost: 0.17014667391777039
Step 0. Cost: 0.14144782721996307
Step 0. Cost: 0.1675858348608017
Step 0. Cost: 0.19819103181362152
Step 0. Cost: 0.10074157267808914
Step 0. Cost: 0.11797767132520676
Step 0. Cost: 0.09662567

In [13]:
fig, ax = plt.subplots( 5, 2, sharex=True, sharey=True )
print( clw.state.h.shape )
for i in range( 5 ):
    for j in range( 2 ):
        output = clw.outputs.rnn_output.eval( feed_dict={clw.input : test_data[33+i*2+j:33+i*2+j+1]}, session=clw.sess )
        #h = clw.state.h.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        ax[i,j].plot(scaler.inverse_transform( output[0,0,:,0,0].reshape( -1, 1 ) ), label='pred' )
        #ax[i,j].plot(h[0,:,0,0], label='h')
        ax[i,j].plot([k-24 for k in range(24)], scaler.inverse_transform( test_data[33+i*2+j,-1,:,0,0].reshape( -1, 1 ) ), label='prev')
        ax[i,j].plot(scaler.inverse_transform( test_labels[33+i*2+j,-1,:,0,0].reshape( -1, 1 ) ), label='true')
        ax[i,j].legend()
plt.show()
plt.savefig( "fall_pred.png" )

(?, 24, 7, 1)


In [16]:
fig, ax = plt.subplots( 5, 2, sharex=True, sharey=True )
station_id = 1
print( clw.state.h.shape )
for i in range( 5 ):
    for j in range( 2 ):
        output = clw.outputs.rnn_output.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        #h = clw.state.h.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        ax[i,j].plot(scaler.inverse_transform( output[0,0,:,station_id,0].reshape( -1, 1 ) ), label='pred' )
        #ax[i,j].plot(h[0,:,0,0], label='h')
        ax[i,j].plot([k-24 for k in range(24)], scaler.inverse_transform( train_data[i*2+j,-1,:,station_id,0].reshape( -1, 1 ) ), label='prev')
        ax[i,j].plot(scaler.inverse_transform( train_labels[i*2+j,-1,:,station_id,0].reshape( -1, 1 ) ), label='true')
        ax[i,j].legend()
plt.show()
plt.savefig( "winter_pred.png" )

(?, 24, 7, 1)


In [15]:
fig, ax = plt.subplots( 5, 2, sharex=True, sharey=True )
print( clw.state.h.shape )
for i in range( 5 ):
    for j in range( 2 ):
        output = clw.outputs.rnn_output.eval( feed_dict={clw.input : train_data[13+i*2+j:13+i*2+j+1]}, session=clw.sess )
        #h = clw.state.h.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        ax[i,j].plot(scaler.inverse_transform( output[0,0,:,0,0].reshape( -1, 1 ) ), label='pred' )
        #ax[i,j].plot(h[0,:,0,0], label='h')
        ax[i,j].plot([k-24 for k in range(24)], scaler.inverse_transform( train_data[13+i*2+j,-1,:,0,0].reshape( -1, 1 ) ), label='prev')
        ax[i,j].plot(scaler.inverse_transform( train_labels[13+i*2+j,-1,:,0,0].reshape( -1, 1 ) ), label='true')
        ax[i,j].legend()
plt.show()
plt.savefig( "spring_pred.png" )

(?, 24, 7, 1)


In [18]:
fig, ax = plt.subplots( 5, 2, sharex=True, sharey=True )
station_id = 1
print( clw.state.h.shape )
for i in range( 5 ):
    for j in range( 2 ):
        output = clw.outputs.rnn_output.eval( feed_dict={clw.input : train_data[25+i*2+j:25+i*2+j+1]}, session=clw.sess )
        #h = clw.state.h.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        ax[i,j].plot(scaler.inverse_transform( output[0,0,:,station_id,0].reshape( -1, 1 ) ), label='pred' )
        #ax[i,j].plot(h[0,:,0,0], label='h')
        ax[i,j].plot([k-24 for k in range(24)], scaler.inverse_transform( train_data[25+i*2+j,-1,:,station_id,0].reshape( -1, 1 ) ), label='prev')
        ax[i,j].plot(scaler.inverse_transform( train_labels[25+i*2+j,-1,:,station_id,0].reshape( -1, 1 ) ), label='true')
        ax[i,j].legend()
plt.show()
plt.savefig( "summer_pred.png" )

(?, 24, 7, 1)


In [17]:
'''
fig, ax = plt.subplots( 5, 2, sharex=True, sharey=True )
print( clw.state.h.shape )
for i in range( 5 ):
    for j in range( 2 ):
        output = clw.outputs.rnn_output.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        h = clw.state.h.eval( feed_dict={clw.input : train_data[i*2+j:i*2+j+1]}, session=clw.sess )
        ax[i,j].plot(output[0,0,:,0,0], label='pred' )
        ax[i,j].plot(h[0,:,0,0], label='h')
        ax[i,j].plot([k-24 for k in range(24)], train_data[i*2+j,-1,:,0,0], label='prev')
        ax[i,j].plot(train_labels[i*2+j,-1,:,0,0], label='true')
        ax[i,j].legend()
plt.show()
'''

(?, 24, 7, 1)


In [4]:
sine_data = ConvLstm.gen_sine(24,24*50)
train_sine, label_sine = create_batch(sine_data, 10, seq_len, time_span, space_size, num_vars)
#label_sine = sine_data[24:24*8].reshape(batch_size, seq_len, time_span, space_size, num_vars)
#train_sine = sine_data[:5*24*7].reshape(5, seq_len, time_span, space_size, num_vars)

In [9]:
temp_outliers = np.abs( temp44 ) < 50
print( np.sum(temp_outliers) )
plt.plot(temp44[temp_outliers])
plt.show()

13184


In [5]:
state = clw.train( train_sine, label_sine, 10000 )

Step 0. Cost: 0.4978961944580078
Step 100. Cost: 0.44712308049201965
Step 200. Cost: 0.38846510648727417
Step 300. Cost: 0.3289157450199127
Step 400. Cost: 0.27411141991615295
Step 500. Cost: 0.2298477441072464
Step 600. Cost: 0.19739758968353271
Step 700. Cost: 0.17376796901226044
Step 800. Cost: 0.15631802380084991
Step 900. Cost: 0.14321015775203705
Step 1000. Cost: 0.1331239938735962
Step 1100. Cost: 0.1251254379749298
Step 1200. Cost: 0.11858422309160233
Step 1300. Cost: 0.11308616399765015
Step 1400. Cost: 0.10835746675729752
Step 1500. Cost: 0.10421194136142731
Step 1600. Cost: 0.1005205288529396
Step 1700. Cost: 0.09719176590442657
Step 1800. Cost: 0.09415720403194427
Step 1900. Cost: 0.0913601964712143
Step 2000. Cost: 0.08874910324811935
Step 2100. Cost: 0.0862775444984436
Step 2200. Cost: 0.08391418308019638
Step 2300. Cost: 0.08165451139211655
Step 2400. Cost: 0.07950732856988907
Step 2500. Cost: 0.07745727151632309
Step 2600. Cost: 0.07545455545186996
Step 2700. Cost: 0.07

In [11]:
plt.plot( train_data[-1,0,:,0,0] )
plt.show()

In [ ]:
plt.plot( train_sine[0,0,:,0,0] )
plt.show()